In [1]:
import sys
sys.path.append("/home/scro4068/miniconda3/envs/prolif/lib/python3.12/site-packages/")

In [2]:
from rdkit import Chem
import prolif as plf
import MDAnalysis as mda
import numpy as np
import pandas as pd

In [3]:
def create_vector(index):
    #rdkit_prot = Chem.MolFromPDBFile(f'../dataset/refined-set/{index}/protein_h.pdb')
    #rdkit_protein_mol = plf.Molecule(rdkit_prot)
    u = mda.Universe(f'../dataset/refined-set/{index}/protein_h.pdb')
    prot = u.select_atoms('protein')
    rdkit_protein_mol = plf.Molecule.from_mda(prot)
    ligand_file = str(f'../dataset/refined-set/{index}/ligand_h.sdf')
    ligand_mol = plf.sdf_supplier(ligand_file)[0]
    # Define the fingerprint generator with a set of consistent interaction types
    interaction_types = ['Hydrophobic', 'HBDonor', 'HBAcceptor', 'PiStacking', 'Anionic', 'Cationic', 'CationPi', 'PiCation', 'VdWContact']

    fpgen = plf.Fingerprint(interaction_types, count=True)

    # Generate the interaction fingerprint
    fingerprint = fpgen.run_from_iterable([ligand_mol], rdkit_protein_mol)

    # Convert the generated fingerprint to a DataFrame
    df = fpgen.to_dataframe()  # No argument is passed here
    df_T = df.T
    df_flat = df_T.droplevel(0, axis=0)
    df_flat = df_flat.reset_index()
    #keep only the first three letters from protein
    df_flat['residue'] = df_flat['protein'].str[:3]
    df_flat
    amino_acids = ['ALA','ARG','ASN','ASP','CYS','GLN','GLU','GLY','HIS','ILE','LEU','LYS','MET','PHE','PRO','SER','THR','TRP','TYR','VAL']
    vector = np.zeros((len(amino_acids),len(interaction_types)))
    for i in range(vector.shape[0]):
        for j in range(vector.shape[1]):
            if interaction_types[j] in df_flat['interaction'].values and amino_acids[i] in df_flat['residue'].values:
                vector[i,j] = df_flat[(df_flat['interaction'] == interaction_types[j]) & (df_flat['residue'] == amino_acids[i])][0].sum()
    
    return vector

In [4]:
features = pd.read_csv('../BFE_ML/feature_calc/features.csv', comment='#', sep=',')

In [5]:
features

,Molecular weight,Acceptors,Donors,LogP,Molecular Refractivity,Wiener index,Ids,PD_amide,PD_positive,PD_neutral,PD_heteroatom,PD_OH,PA_amide,PA_negative,PA_neutral,PA_aromatic,Free energy,P_logp,P_MR
0,231.170319,2.0,3.0,-0.13160,60.5356,461.0,2tpi,24.0,0.0,0.0,0.0,5.0,23.0,5.0,4.0,1.0,-5.874825,-0.665612,29.440791
1,147.112804,2.0,3.0,-0.85160,36.0603,130.0,4tln,6.0,9.0,4.0,0.0,2.0,6.0,6.0,4.0,7.0,-5.072544,-0.403714,34.962933
2,363.057999,10.0,6.0,-2.56970,76.4808,1230.0,1rnt,9.0,6.0,6.0,0.0,3.0,9.0,9.0,10.0,7.0,-7.070675,-0.619947,36.111405
3,182.081170,2.0,3.0,-0.37020,46.1227,268.0,4ts1,11.0,4.0,5.0,1.0,4.0,12.0,11.0,10.0,5.0,-6.727787,-0.498970,31.695033
4,519.213437,5.0,5.0,3.26060,135.1412,4249.0,4tmn,16.0,9.0,5.0,1.0,6.0,16.0,13.0,10.0,9.0,-13.859616,-0.503965,34.127585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5311,253.050394,3.0,2.0,1.21620,64.9371,478.0,6d1i,6.0,9.0,5.0,0.0,0.0,5.0,4.0,2.0,8.0,-5.577268,-0.570889,33.634874
5312,212.025563,3.0,1.0,0.53470,50.5625,260.0,6uh0,5.0,7.0,6.0,1.0,3.0,6.0,5.0,5.0,8.0,-8.272838,-0.393956,34.876296
5313,404.151622,6.0,1.0,4.72902,114.4347,2134.0,6k04,10.0,2.0,2.0,1.0,2.0,9.0,3.0,4.0,4.0,-10.960875,-0.186890,36.583310
5314,436.991713,8.0,5.0,0.42410,89.9438,1701.0,6ic2,8.0,7.0,6.0,1.0,3.0,10.0,4.0,4.0,8.0,-8.229820,-0.330392,33.531504


In [6]:
indices = features['Ids']

In [7]:
'''vectorlist = []
with open('vectors.csv', 'w') as f:
    f.write('index,vector\n')
    for index in indices:
        createdvector= create_vector(index).flatten()
        vectorlist.append(createdvector)
        #append to file after each iteration
        f.write(f'{index},{createdvector}\n')
'''

"vectorlist = []\nwith open('vectors.csv', 'w') as f:\n    f.write('index,vector\n')\n    for index in indices:\n        createdvector= create_vector(index).flatten()\n        vectorlist.append(createdvector)\n        #append to file after each iteration\n        f.write(f'{index},{createdvector}\n')\n"

In [8]:
#Read vectors.csv
vector_csv = []
index_list = []
with open('vectors.csv', 'r') as f:
    n = 0
    for line in f:
        if n == 0:
            n+=1
            continue
        if n == 1:
            vect = ""
            spl = line.split(',')
            index_list.append(spl[0])
            vect += spl[1][1:-2]+'.'
            n+=1
            continue            
        if "," in line:
            vector_csv.append(vect)
            vect = ""
            spl = line.split(',')
            index_list.append(spl[0])
            vect += spl[1][1:-2]+"."
            n+=1
        else:
            vect += line[1:-2]+"."
            n+=1
    vector_csv.append(vect)


In [9]:
vector_clean = []
for j in range(len(vector_csv)):
    tmp = []
    for i,v in enumerate(vector_csv[j].split('.')[:-2]):
        try:
            tmp.append(int(v))
        except:
            try:
                tmp.append(int(v[1:]))
            except:
                print(v)
                tmp.append(int(v[2:]))
    vector_clean.append(tmp)

In [10]:
len(vector_clean)

3439

In [11]:
import gc

In [12]:
#Continue from the last written index
start = index_list[-1]
print(start)
start_idx = np.where(indices == start)[0][0]
print(start_idx, indices[start_idx+1])
skip = '4o09'
for index in indices[start_idx+1:]:
    f = open('vectors.csv', 'a')
    if index == skip:
        f.write(f"{skip}, [NaN]\n")
        continue
    createdvector = create_vector(index).flatten()

    #append to file after each iteration
    f.write(f'{index},{createdvector}\n')
    f.close()
    #empty the memory
    del createdvector
    gc.collect()
#f.close()

4umc
3438 4o05


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

: 

: 

: 

In [ ]:
#Make a dictionary with the index as key and the vector as value
vector_dict = dict(zip(index_list, vector_clean))
vector_df   = pd.DataFrame(vector_dict).T

In [ ]:
vector_df

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
2tpi,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4tln,0,0,0,0,0,0,0,0,3,0,...,0,1,0,0,0,0,0,0,0,0
1rnt,0,0,0,0,0,0,0,0,0,0,...,7,0,0,0,0,0,0,0,0,0
4ts1,0,0,0,0,0,0,0,0,0,0,...,5,0,0,0,0,0,0,0,0,0
4tmn,0,1,0,0,0,0,0,0,3,0,...,2,4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1sqt,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
1v2k,3,0,0,0,0,0,0,0,2,0,...,3,2,0,0,0,0,0,0,0,1
1ta6,2,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,1
1u0g,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
